In [12]:
from ibm_quantum_widgets import CircuitComposer

from qiskit.quantum_info import Statevector, Pauli
from sympy.physics.quantum import pauli

from qiskit.visualization import plot_bloch_multivector

from qiskit.circuit.library import EfficientSU2


from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, Aer, transpile
from numpy import pi
import numpy as np
from qiskit.quantum_info.operators import Operator, Pauli, predicates

In [13]:
Aer.backends()
simulator = Aer.get_backend('aer_simulator')

qreg_q = QuantumRegister(1, 'q')
circuit = QuantumCircuit(qreg_q)

clreg = ClassicalRegister(1)

sx = pauli.SigmaX()
sy = pauli.SigmaY()
sz = pauli.SigmaZ()

In [14]:
def to_spherical(state):
    r0 = np.abs(state[0])
    ϕ0 = np.angle(state[0])
    r1 = np.abs(state[1])
    ϕ1 = np.angle(state[1])
    r = np.sqrt(r0 ** 2 + r1 ** 2)
    θ = 2 * np.arccos(r0 / r)
    ϕ = ϕ1 - ϕ0
    return [r, θ, ϕ]

def to_cartesian(polar):
    r = polar[0]
    θ = polar[1]
    ϕ = polar[2]
    x = r * np.sin(θ) * np.cos(ϕ)
    y = r * np.sin(θ) * np.sin(ϕ)
    z = r * np.cos(θ)
    return [x, y, z]

Use of Operator-class: https://qiskit.org/documentation/tutorials/circuits_advanced/02_operators_overview.html#Operator-Class

In [23]:
def rn_su2_1(theta, n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    return ([
    [np.cos(n1/2.0)*np.exp(1j/2.0*(n2+n3)), np.sin(n1/2)*np.exp(-(1j/2.0)+(n2-n3))],
    [-np.sin(n1/2.0)*np.exp(1j/2.0*(n2-n3)), np.cos(n1/2)*np.exp(-1j/2.0*(n2+n3))]
    #ToDo: find unitary SU(2) matrix
    ])

#https://link.springer.com/content/pdf/bbm%3A978-3-540-29082-7%2F1.pdf

print(rn_su2_1(pi,1,1,1))

circuit = transpile(circuit, simulator)

[[(0.4741598817790379+0.7384602626041288j), (0.42073549240394825-0.22984884706593015j)], [(-0.479425538604203+0j), (0.4741598817790379-0.7384602626041288j)]]


In [86]:
def rn_su2_1a(theta, n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    return Operator([
    [np.cos(n1/2.0)*np.exp(1j/2.0*(n2+n3)), np.sin(n1/2)*np.exp(-(1j/2.0)+(n2-n3))],
    [-np.sin(n1/2.0)*np.exp(1j/2.0*(n2-n3)), np.cos(n1/2)*np.exp(-1j/2.0*(n2+n3))]
    #ToDo: find unitary SU(2) matrix
    ])

print(rn_su2_1a(pi,1,1,1))
#rotated = circuit.unitary(rn_su2_1a(pi,1,np.pi,0.5),1)

#https://link.springer.com/content/pdf/bbm%3A978-3-540-29082-7%2F1.pdf

Operator([[ 0.47415988+0.73846026j,  0.42073549-0.22984885j],
          [-0.47942554+0.j        ,  0.47415988-0.73846026j]],
         input_dims=(2,), output_dims=(2,))


In [ ]:
def rn_su2_2(n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    op = ([
    [np.exp(1j+(n1+n2)/2)+np.cos(0.5+n3) , 1j+np.exp(1j*(n1-n2)/2)*np.sin(0.5+n3)],
    [1j+np.exp(1j*(n2-n1)/2)*np.sin(0.5*n3) , np.exp(-1j*(n1+n2)/2)*np.cos(0.5*n3)]
    #ToDo: find unitary SU(2) matrix
    ])
    return op

#  0 <= n1 < 2pi
#  0 <= n3 < pi
# -2pi <= n2 < 2pi

print(rn_su2_2(1,np.pi,0.5))
rotated = circuit.unitary(rn_su2_2(1,np.pi,0.5),1)

circuit.append(rn_su2_2(1,np.pi,0.5))
circuit.draw


In [ ]:
def rn_su2_3(theta, n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    return Operator([
    [np.cos(theta/2)- 1j*n3*np.sin(theta/2), -np.sin(theta/2)*(n2+1j*n1)],
    [np.sin(theta/2)*(n2-1j*n1), np.cos(theta/2)+1j*n3*np.sin(theta/2)]
    ])

print(rn_su2_3(1,1,np.pi,0.5))
#rotated = circuit.unitary(rn_su2_2(pi,1,np.pi,0.5),1)


In [19]:
def rn_su2_4(theta, n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    return Operator([
    [np.cos(theta/2)+ 1j*n3*np.sin(theta/2), -1j*(n1-1j*n2)*np.sin(theta/2)],
    [-1j*(n1+1j*n2)*np.sin(theta/2), np.cos(theta/2)+1j*n3*np.sin(theta/2)]
    ])

print(rn_su2_4(1,1,np.pi,0.5))
rotated = predicates.is_unitary_matrix(rn_su2_4(1,1,np.pi,0.5))

#rotated = circuit.unitary(rn_su2_4(np.pi,1,np.pi,0.5),1)

print(rotated)
#https://www.uni-muenster.de/Physik.TP/archive/fileadmin/lehre/teilchen/ws1011/SO3SU2.pdf

Operator([[ 0.87758256+0.23971277j, -1.50615975-0.47942554j],
          [ 1.50615975-0.47942554j,  0.87758256+0.23971277j]],
         input_dims=(2,), output_dims=(2,))
False


In [11]:
def rn_su2_5(theta, n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    return Operator([
    [np.cos(theta/2)- 1j*n3*np.sin(theta/2), -1j*(n1-1j*n2)*np.sin(theta/2)],
    [-1j*(n1+1j*n2)*np.sin(theta/2), np.cos(theta/2)+1j*n3*np.sin(theta/2)]
    ],input_dims=(2, 1), output_dims=(2, 1))

print(rn_su2_5(pi,1,1,1))

unitary = is_uni

rotated = circuit.unitary(rn_su2_5(pi,1,1,1),1)

#Quelle: https://docplayer.org/117986458-Die-symmetriegruppen-so-3-und-su-2.html


Operator([[ 6.123234e-17-1.j, -1.000000e+00-1.j],
          [ 1.000000e+00-1.j,  6.123234e-17+1.j]],
         input_dims=(2, 1), output_dims=(2, 1))


TypeError: must be real number, not method

In [ ]:
def rn_su2_6(theta, n1,n2,n3):
    #This represents a matrix operator that will evolve() a Statevector by matrix-vector multiplication and will evolve() a DensityMatrix by left and right multiplication
    return ([
    [np.cos(theta/2)- 1j*n3*np.sin(theta/2), -1j*(n1-1j*n2)*np.sin(theta/2)],
    [-1j*(n1+1j*n2)*np.sin(theta/2), np.cos(theta/2)+1j*n3*np.sin(theta/2)]
    ])

print(rn_su2_6(pi,1,1,1))



efcircuit = EfficientSU2(1,rn_su2_6(pi,1,1,1))

#Quelle: https://qiskit.org/documentation/stubs/qiskit.circuit.library.EfficientSU2.html


In [ ]:
states = []
points = []

alpha = 1/np.sqrt(2)
beta = 1/np.sqrt(2)
s = np.array([alpha,beta])
state = Statevector(s)
states.append(state)

Ψ = [complex(alpha, 0), complex(beta, 0)]
polar = to_spherical(Ψ)
pnt = to_cartesian(polar)
points.append(pnt)

rotated = state

for i in range(0,10):

    #Apply unitary gate specified by obj to qubits.
    rotated = circuit.unitary(rn_su2_2(pi,1,np.pi,0.5),1)


    pnt = [(sx*rotated), (sy*rotated), (sz*rotated)]

    #ToDo: patch function
    r = np.sqrt(pnt[0]**2+pnt[1]**2+pnt[2]**2)

    states.append(rotated)
    points.append(pnt)

circuit.add_bits(points)